In [4]:
import tensorflow as tf
import numpy as np

In [5]:
import pickle

In [6]:
with open('../X_smallset.pkl', 'rb') as inp:
    inputs = pickle.load(inp)

In [7]:
single_input = inputs[0]

In [8]:
test_input = [np.array([[0, 1, 1, 0, 0, 1.5],
             [1, 0, 0, 1, 0, 1.2],
             [1, 0, 0, 1, 0, 1.4]]),
             [[[1, [0, 0, 1]], [2, [0, 1, 0]]],
             [[0, [0, 0, 1]]],
             [[0, [0, 1, 0]]]],
             np.array([3, 2, 4, 7])]

In [9]:
features = single_input[0]
adjacency_list = single_input[1]
physics_features = single_input[2]

In [11]:
w_s = tf.Variable(initial_value=tf.initializers.glorot_uniform()(shape=[81, 80], dtype='double'),shape=[81, 80])
w_n = tf.Variable(initial_value=tf.initializers.glorot_uniform()(shape=[103, 80], dtype='double'),shape=[103, 80])

In [12]:
features[0][-3:]

array([0.031, 1.55 , 0.452])

In [13]:
combination_orders = ([[0, -6], tf.math.add], [[-6, -3], 'position'], [[-3], tf.multiply])

In [14]:
def AtomDistance(x, y):
    return tf.sqrt(tf.reduce_sum(tf.square(x - y)))

In [15]:
new_features = len(features)*[None]
for i, adj in enumerate(adjacency_list):
    self_features = tf.reshape(features[i], [1, 81])
    self_conv_features = tf.matmul(self_features, w_s)
    new_features[i] = self_conv_features
    for neighbour in adj:
        neighbour_features = features[neighbour[0]]
        neighbour_bond = neighbour[1]
        new_ordered_features = []
        distance = None
        for j, order in enumerate(combination_orders):
            order_function = order[1]
            indices = order[0]
            if j == len(combination_orders) - 1 and len(indices) == 1:
                new_ordered_features.append(order_function(self_features[0][indices[0]:], 
                                                                 neighbour_features[indices[0]:]))
            else:
                if order_function == 'position':
                    distance = AtomDistance(self_features[0][indices[0]:indices[1]], 
                                                                   neighbour_features[indices[0]:indices[1]])
                    new_ordered_features.append(neighbour_features[indices[0]:indices[1]])
                else:
                    new_ordered_features.append(order_function(self_features[0][indices[0]:indices[1]], 
                                                                   neighbour_features[indices[0]:indices[1]]))
        new_ordered_features = tf.concat(new_ordered_features, axis=0)
        if distance != None:
            new_ordered_features /= distance**2
        new_ordered_features = tf.concat([new_ordered_features, neighbour_bond], axis=0)
        new_ordered_features = tf.reshape(new_ordered_features, [1, 103])
        new_features[i] += tf.matmul(new_ordered_features, w_n)

In [49]:
import tensorflow as tf

class MultiGraphConvLayer(tf.keras.Model):
    def __init__(self,
                 out_channel,
                 num_features=81,
                 num_bond=22,
                 activation_fn=None,
                 combination_rules = []
                 ):
        super(MultiGraphConvLayer, self).__init__()
        self.out_channel = out_channel
        self.num_features = num_features
        self.num_bond = num_bond
        self.activation_fn = activation_fn
        self.combination_rules = combination_rules
        self.w_s = tf.Variable(initial_value=tf.initializers.glorot_uniform()
        (shape=[num_features, out_channel]), shape=[num_features, out_channel], trainable=True)
        self.w_n = tf.Variable(initial_value=tf.initializers.glorot_uniform()
        (shape=[num_features + num_bond, out_channel]),
                               shape=[num_features + num_bond, out_channel], trainable=True)

    def AtomDistance(self, x, y):
        return tf.sqrt(tf.reduce_sum(tf.square(x - y)))

    def addRule(self, rule, start_index, end_index = None):
        rules_dict = {
            "sum": tf.math.add,
            "multiply": tf.math.multiply,
            "distance": "distance",
            "divide": tf.math.divide,
            "subtract": tf.math.subtract,
        }
        if type(rule) == str:
            rule = rules_dict[rule]
        if end_index == None:
            self.combination_rules.append([[start_index], rule])
        else:
            self.combination_rules.append([[start_index, end_index], rule])

    def _call_single(self, inp):
        features = inp[0]
        adjacency_list = inp[1]
        new_features = len(features) * [None]
        for i, adj in enumerate(adjacency_list):
            self_features = tf.reshape(features[i], [1, self.num_features])
            self_conv_features = tf.matmul(self_features, self.w_s)
            new_features[i] = self_conv_features
            for neighbour in adj:
                neighbour_features = features[neighbour[0]]
                neighbour_bond = neighbour[1]
                new_ordered_features = []
                distance = None
                for j, rule in enumerate(self.combination_rules):
                    rule_function = rule[1]
                    indices = rule[0]
                    if j == len(self.combination_rules) - 1 and len(indices) == 1:
                        new_ordered_features.append(rule_function(self_features[0][indices[0]:],
                                                                  neighbour_features[indices[0]:]))
                    else:
                        if rule_function == 'distance':
                            distance = self.AtomDistance(x=self_features[0][indices[0]:indices[1]],
                                                    y=neighbour_features[indices[0]:indices[1]])
                            new_ordered_features.append(neighbour_features[indices[0]:indices[1]])
                        else:
                            new_ordered_features.append(rule_function(self_features[0][indices[0]:indices[1]],
                                                                      neighbour_features[indices[0]:indices[1]]))
                new_ordered_features = tf.concat(new_ordered_features, axis=0)
                if distance != None:
                    new_ordered_features /= distance ** 2
                new_ordered_features = tf.concat([new_ordered_features, neighbour_bond], axis=0)
                new_ordered_features = tf.reshape(new_ordered_features, [1, self.num_features + self.num_bond])
                new_features[i] += tf.matmul(new_ordered_features, self.w_n)
                new_features[i] = new_features[i][0]
        return ([new_features, adjacency_list, inp[2:]])

    def call(self, inputs):
        output = []
        for inp in inputs:
            output.append(self._call_single(inp))
        return output

In [50]:
combination_orders = ([[0, -6], tf.math.add], [[-6, -3], 'distance'], [[-3], tf.multiply])
m = MultiGraphConvLayer(80)
m.addRule("sum", 0, -6)
m.addRule("distance", -6, -3)
m.addRule("multiply", -3)

In [51]:
outs = m(inputs)

TypeError: Exception encountered when calling layer "multi_graph_conv_layer_7" (type MultiGraphConvLayer).

'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment

Call arguments received:
  • inputs=[['tf.Tensor(shape=(1724, 81), dtype=float32)', [[['tf.Tensor(shape=(), dtype=int32)', 'tf.Tensor(shape=(22,), dtype=float32)']], [['tf.Tensor(shape=(), dtype=int32)', 'tf.Tensor(shape=(22,), dtype=float32)'], ['tf.Tensor(shape=(), dtype=int32)', 'tf.Tensor(shape=(22,), dtype=float32)'], ['tf.Tensor(shape=(), dtype=int32)', 'tf.Tensor(shape=(22,), dtype=float32)']], [['tf.Tensor(shape=(), dtype=int32)', 'tf.Tensor(shape=(22,), dtype=float32)'], ['tf.Tensor(shape=(), dtype=int32)', 'tf.Tensor(shape=(22,), dtype=float32)'], ['tf.Tensor(shape=(), dtype=int32)', 'tf.Tensor(shape=(22,), dtype=float32)']], [['tf.Tensor(shape=(), dtype=int32)', 'tf.Tensor(shape=(22,), dtype=float32)']], [['tf.Tensor(shape=(), dtype=int32)', 'tf.Tensor(shape=(22,), dtype=float32)'], 

In [45]:
outs[0][0][0]

<tf.Tensor: shape=(1, 80), dtype=float32, numpy=
array([[  3.6744733 ,   4.7053304 , -12.801747  ,  -4.2651873 ,
          6.4662557 ,  -3.4161422 ,  -1.1624657 ,   0.07058808,
          0.24753284, -11.887071  ,  10.0788555 ,   4.313283  ,
          9.26334   ,  10.176142  ,   0.33819342,   1.5968744 ,
          8.915068  ,   5.388324  ,  -6.84422   , -18.425306  ,
          9.038089  , -21.085253  ,  13.364895  ,  -7.980207  ,
         -7.628983  ,   2.4417124 ,  12.124406  , -11.393864  ,
        -13.751121  ,   1.5200927 ,  -2.446978  ,   0.9206333 ,
        -11.458265  ,  -0.38857305,   0.43586048,   5.3919106 ,
         -7.583773  , -13.64183   ,   6.510283  , -13.717955  ,
         -8.367491  ,  -8.942728  ,   2.5730126 ,   0.6580807 ,
         -5.057567  ,   1.9755421 ,  -5.8821707 , -18.61711   ,
          3.1775973 ,   8.676688  ,  -0.10977328,   0.4908085 ,
         -3.8996706 ,   1.3870703 ,  15.358127  ,   4.3272743 ,
        -10.366617  ,  -6.427205  ,   3.3024514 ,   9.0